# <center>  <span style='color:#f1552e'> Forklift's Economic Life Prediction - Modelling Approach III -Update2

# <span style='color:#f1552e'> Import libraries

In [1]:
# load Libraries (packages should be installed prior to importing)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact, Layout
import shapely
from shapely.geometry import LineString
from IPython.core.display import display, HTML, clear_output
import warnings
warnings.filterwarnings("ignore")

In [32]:
#pip install shapely

# <span style='color:#f1552e'> Define Functions

## Load Dataset

In [2]:
# function to read raw dataset file & filter toyota forklifts
def load():
    # read dataset from current working directory
    workOrders = pd.read_csv(r"WOs.csv", encoding="cp1252")

    # apply filters on dataset
    toyo_forklifts = workOrders.loc[
        (workOrders["Make"] == "TOYO")  # filter Toyota forklifts
        & (
            workOrders["Model"].str[2] == "B"
        )  # filter Electric (battery operated) forlifts
        & (workOrders["FLS GRP"] == "FORKLIFTS")  # filter forklifts only
    ]

    # get list of equipments
    equip_list = list(toyo_forklifts["Equipment Object"].unique())

    # get list of models
    model_list = list(toyo_forklifts["Model"].unique())

    # filter dataset by model and equipments (since Make is blank for some Toyota electric forklifts)
    toyo_fl = workOrders.loc[
        (workOrders["Model"].isin(model_list))
        & (workOrders["Equipment Object"].isin(equip_list))
    ]

    return toyo_fl

## Clean Dataset

In [3]:
# function to clean the dataset
def clean(df):
    # convert date columns to type date
    df[
        ["Posting Date", "Order Date", "Time Sheet Last Date", "Time Sheet First Date"]
    ] = pd.to_datetime(
        df[
            [
                "Posting Date",
                "Order Date",
                "Time Sheet First Date",
                "Time Sheet Last Date",
            ]
        ].stack()
    ).unstack()

    # remove $ and , from all cost columns
    df[
        [
            "Amount",
            "PM",
            "REPAIR",
            "TIRES",
            "DAMAGE",
            "REQUEST",
            "ATTACH",
            "Key On Meter",
        ]
    ] = (
        df[
            [
                "Amount",
                "PM",
                "REPAIR",
                "TIRES",
                "DAMAGE",
                "REQUEST",
                "ATTACH",
                "Key On Meter",
            ]
        ]
        .stack()
        .replace("[$,]", "", regex=True)
        .astype(float)
        .unstack()
    )

    # fill null order date
    df.loc[df["Order Date"].isnull(), "Order Date"] = df.loc[
        df["Order Date"].isnull(), "Posting Date"
    ]

    # update model for equipment with 2 model numbers
    df.loc[df["Equipment Object"] == "50071", "Model"] = "7FBCU25"

    return df

## Filter Dataset

In [4]:
# function to remove WOs not required for modelling
def filter(df):
    # remove service types where key on meter or cost is 0
    service_remove = [
        "NDEMO",
        "AUDIT-CUST",
        "CABUSE",
        "E-PARTS",
        "MAN",
        "NAFTER-JB",
        "NAFTERS",
        "NPDI",
        "NPDITM",
        "ROLLED",
    ]
    df = df[-df["Service Type"].isin(service_remove)]

    # remove equipment where Key on meter is 0 / incorrect
    equip_remove = ["40305", "40308", "E051013", "E052406", "Rental-5535"]
    df = df[-df["Equipment Object"].isin(equip_remove)]

    # remove WO w/o Amount
    df = df.loc[(df.Amount.notna())]

    # remove WO w/o Key on meter
    df = df[df["Key On Meter"].notnull()]

    # filter repair, PM & damage WO
    df = (
        pd.concat(
            [df[(df["REPAIR"] > 0)], df[(df["PM"] > 0)], df[(df["DAMAGE"] > 0)]],
            ignore_index=True,
        )
        .drop_duplicates()
        .reset_index(drop=True)
    )

    return df

## Dataset Details

In [5]:
# function to view basic details about dataset
def info(df):
    print("\033[1;34m")
    print(
        "Number Of Columns in Cleaned Dataset:",
        "\033[0m",
        df.shape[1],
    )
    print("\033[1;34m")
    print("Number Of Rows in Cleaned Dataset:", "\033[0m", df.shape[0])

    print("\033[1;34m")
    print("Datatype:", "\033[0m")
    df.info(verbose=True)
    
    display(HTML('<h2> Cleaned Dataset (First 5 Rows)</h2>'))

    print("\033[1;34m")
    print("Cleaned Dataset (First 5 Rows)", "\033[0m")
    display(df.head(5))

## Transform Data

In [6]:
# function to transform the dataset for modelling
def transform_bin(df):
   
    # define upper limit for key on meter binning
    upper_limit = int(round(df["Key On Meter"].max(), -3))
   
    # create intervals
    KOM_interval = pd.interval_range(start=0, freq=1000, end=upper_limit)
    
    # assign bins to WO based on Key on Meter (KOM bin refer to upper limit of bin)
    df["KOM Bin"] = (
        pd.cut(df["Key On Meter"], bins=KOM_interval, include_lowest=True)
        .astype("string")
        .str.split(" ")
        .str[1]
        .str.split("]")
        .str[0]
        .fillna(0)
        .astype(float)
        .astype(int)
    )
    
    # group records by equipment Object & bin and add damage, pm, repair and actual hours
    df = (
        df.groupby(["Equipment Object", "KOM Bin","Model"])
        .agg(
            {
                "DAMAGE": ["sum"],
                "PM": ["sum"],
                "REPAIR": ["sum"],
                "Actual Hours": ["sum"],
            }
        )
        .reset_index()
    )
    print(df)
    # rename dataframe columns
    df.columns = df.columns.droplevel()
    df.columns = [
        "Equipment",
        "KOM Bin",
        "Model",
        "Total Damage",
        "Total PM",
        "Total Repair",
        "Total Actual Hours",
    ]

    # print size of dataset
    print("\033[1;34m")
    print("Number Of Rows in Transformed Dataset::", "\033[0m", df.shape[0])
    print("\033[1;34m")
    print("Number Of Columns in Transformed Dataset::", "\033[0m", df.shape[1])
    print("\033[1;34m", "\nTransformed Dataset (First 5 Rows)", "\033[0m")

    # View transformed data
    display(df.head(5))

    return df

## Models

In [7]:
# function for linear regression model - scikit
def linreg(df):
    # predictor variables from dataset
    X = df[["KOM Bin", "Total Damage", "Total PM", "Total Actual Hours"]]
    # Target variable
    y = df["Total Repair"]

    # split X & y into train & test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=83
    )

    # create linear regression model and fit the model on training data
    linreg = LinearRegression().fit(X_train, y_train)

    # make predictions on test data using linear regression model
    y_pred = linreg.predict(X_test)

    # calculate R-squared
    score = round(r2_score(y_test, y_pred), 4)

    # print results
    print(
        "\033[1;34m",
        "\nR2 of Linear Regression Model:",
        "\033[0m",
        score,
    )
    print("\033[1;34m", "\nIntercept:", "\033[0m", linreg.intercept_)
    print("\033[1;34m", "\nCoefficients:", "\033[0m", linreg.coef_)
    print(
        "\033[1;34m",
        "\nLinear Regression Equation:\n",
        "\033[0m",
        "Repair = %0.4f + (%0.4f * KOM Bin) + (%0.4f * Damage) + (%0.4f * PM) + (%0.4f * Actual Hours)"
        % (
            linreg.intercept_,
            linreg.coef_[0],
            linreg.coef_[1],
            linreg.coef_[2],
            linreg.coef_[3],
        ),
    )
    
    # make predictions on entire dataset using linear regression model
    y_pred = linreg.predict(X)
    
    pred_df = df[["Equipment",'KOM Bin', "Model",'Total Damage', 'Total PM','Total Actual Hours']]
    pred_df["Actual Repair"] = df["Total Repair"]/1000
    pred_df["Predicted Repair"] = pd.DataFrame(y_pred)[0]/1000
    
    return linreg, pred_df, score

In [8]:
# function for linear regression model - statsmodel
def olsreg(df):
    # predictor variables from dataset
    X = df[["KOM Bin", "Total Damage", "Total PM", "Total Actual Hours"]]

    # Target variable
    y = df["Total Repair"]

    # split X & y into train & test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=46
    )

    # create linear regression model and fit the model on training data
    olsreg = sm.OLS(y_train, sm.add_constant(X_train)).fit()
    print(olsreg.summary())

## Plots

In [9]:
# plot to determine intersection point of repair and acquistion cost 
# Economic Life prediction plot
def plot(Equipment,Acquistion):
    # filter dataset by equipment
    plot = pred_df[pred_df['Equipment'] == Equipment]
    Model = plot["Model"].unique()
    
    # new plot
    fig = go.Figure()
    
    # add trace for predicted repair cost per hour
    fig.add_trace(
        go.Scatter(
            x=plot['KOM Bin'],
            y=plot['Predicted Repair'],
            name="Repair(Pred)",
            mode="lines",
            marker_color="blue",
            hovertemplate =
                            '<br><i>Key On Meter</i> : %{x}<br>' +
                            '<i>Pred Repair Cost/Hr</i> : $%{y:.2f}'
        )
    )
    
    # add trace for acquistion cost per hour
    fig.add_trace(
        go.Scatter(
            x=plot['KOM Bin'],
            y=Acquistion/plot['KOM Bin'],
            name="Acquisition",
            mode="lines",
            marker_color="red",
            hovertemplate =
                            '<br><i>Key On Meter</i> : %{x}<br>'+
                            '<i>Acq Cost/Hr</i> : $%{y:.2f}')
    )
    
    # find intersection points
    if plot.shape[0] > 1:
        line_1 = LineString(np.column_stack((plot['KOM Bin'],plot['Predicted Repair'])))
        line_2 = LineString(np.column_stack((plot['KOM Bin'],Acquistion/plot['KOM Bin'])))
        intersection = line_1.intersection(line_2)
    
        # plot intersection point if it exists
        if intersection.is_empty:
            pass
        else:
            # get the intersection point type
            point_type = str(type(intersection)).split(".")[2]
            # get intersection points if type is multipoint
            if (point_type == "multipoint"):
                xs = [point.x for point in intersection]
                ys = [point.y for point in intersection]
            # get intersection points if type is point
            else:
                xs = np.array(intersection.x)
                ys = np.array(intersection.y)
            
            # plot intersection points
            fig.add_trace(
                go.Scatter(
                    x=xs,
                    y=ys,
                    name="Economic Life",
                    mode="markers",
                    marker=dict(symbol = 'diamond',
                                color='skyblue',
                                size=10,
                                line=dict(color='#A22179',
                                          width=2,
                                        )),
                                hovertemplate =
                            '<br><i>Key On Meter</i> : %{x:.0f}' +
                            '<br><i>Cost/Hr</i> : $%{y:.2f}'),

    )
    
    # update plot layout
    fig.update_layout(
        #hovermode="x unified",
        legend=dict(
            x=0.82,
            y=1),
        xaxis=dict(mirror = True ,
                    title= "Key On Meters(Hrs)"
                    ),
        yaxis=dict(mirror =True,
        title="Cost/Hr($)"
                ) ,

        template="simple_white",
        yaxis_tickformat = '$',
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        title={
            "text": "Acquisition Vs Repair Cost <br> (Equipment: " + str(Equipment) +", Model: "+ str(Model[0]) +")",
            "y": 0.9,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )
    
    # display plot
    fig.show()

In [10]:
# repair cost prediction interactive layout
# create input text for all predictors
KOM = widgets.Text(description='KeyOnMeter')
Damage = widgets.Text(description='Damage')
PM = widgets.Text(description='PM')
Hours = widgets.Text(description='Hours')
    
# create buttons for actions
button = widgets.Button(description="Predict")
reset_button = widgets.Button(description = "Reset")
output = widgets.Output()
    
# define layout
widget_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%')
    
# design display box
Box = widgets.VBox([widgets.HBox([KOM, PM]), 
                        widgets.HBox([Damage,Hours]), widgets.HBox([button, reset_button])],layout=widget_layout)

# define predict button action
def on_button_clicked(b):
    with output:
        clear_output()
        if not KOM.value:
            error_popup()
        else:
            result = round(model.predict([[float(KOM.value), float(Damage.value), float(PM.value), float(Hours.value)]])[0], 2 )
            print("\033[1;32m","Predicted Repair Cost:", result)

# define reset action
def reset_values(b):
    KOM.value, Damage.value,PM.value, Hours.value  = ["" for x in range(4)]
    with output:
        clear_output()

# assign on click action
button.on_click(on_button_clicked)
reset_button.on_click(reset_values)

# <span style='color:#f1552e'> Execution

In [11]:
# Load dataset with toyota forklift WOs
toyo_WO = load()

# clean dataset
toyo_clean = clean(toyo_WO)

# filter dataset
toyo_df = filter(toyo_clean)

# dataset info
info(toyo_df)


Number Of Columns in Cleaned Dataset:  31

Number Of Rows in Cleaned Dataset:  6383

Datatype: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6383 entries, 0 to 6382
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Posting Date           6383 non-null   datetime64[ns]
 1   No.                    6383 non-null   object        
 2   Equipment Object       6383 non-null   object        
 3   Owner                  6383 non-null   object        
 4   Sell-To Customer No.   6383 non-null   object        
 5   Mx Contract            2804 non-null   object        
 6   Fleet                  6093 non-null   object        
 7   Make                   6244 non-null   object        
 8   Model                  6383 non-null   object        
 9   Serial No.             6383 non-null   object        
 10  Service Type           6383 non-null   object        
 11  Service Description    6


Cleaned Dataset (First 5 Rows) 


,Posting Date,No.,Equipment Object,Owner,Sell-To Customer No.,Mx Contract,Fleet,Make,Model,Serial No.,...,DAMAGE,REQUEST,ATTACH,BILL TO,PMs,DT,Days Down,CAT,WC,FLS GRP
0,2008-06-30,WO-002661,40310,Sold,22400-0002,NaN,NaN,NaN,7FBCU25,7FBCU25-61674,...,NaN,NaN,NaN,CUST,NaN,1.5,0.05,REPAIR,UMATILLA,FORKLIFTS
1,2008-06-30,WO-002660,50071,Sold,22400-0002,NaN,NaN,NaN,7FBCU25,7FBCU25-65776,...,NaN,NaN,NaN,CUST,NaN,1.5,0.05,REPAIR,UMATILLA,FORKLIFTS
2,2008-07-10,WO-004297,62026,Sold,22400-0001,NaN,NaN,NaN,7FBCU32,7FBCU32-63297,...,NaN,NaN,NaN,INT,NaN,1.2,0.04,REPAIR,3174,FORKLIFTS
3,2008-07-17,WO-004594,62027,Sold,22400-0001,NaN,NaN,NaN,7FBCU30,7FBCU30-63302,...,NaN,NaN,NaN,CUST,NaN,1.3,0.04,REPAIR,4203,FORKLIFTS
4,2008-07-23,WO-004544,62491,Stock,22400-0002,NaN,NaN,NaN,7FBEU18,7FBEU18-18767,...,NaN,NaN,NaN,INT,NaN,1.5,0.06,REPAIR,UMATILLA,FORKLIFTS


In [14]:
# Transform dataset as required for modelling
toyo_bin_df = transform_bin(toyo_df)

     Equipment Object KOM Bin    Model   DAMAGE       PM   REPAIR Actual Hours
                                            sum      sum      sum          sum
0               38446    4000  7FBCU15   731.60     0.00     0.00          3.3
1               38446    5000  7FBCU15   972.24  1538.94  1496.18         29.9
2               38446    6000  7FBCU15  1106.35  2164.87  3624.66         49.6
3               38446    7000  7FBCU15     0.00  1969.76  3594.30         52.6
4               38446    8000  7FBCU15     0.00   140.58  2137.28         15.3
...               ...     ...      ...      ...      ...      ...          ...
1211          E045173    1000  8FBCU32     0.00   440.57   303.88          7.5
1212          E045173    2000  8FBCU32     0.00   308.72   161.00          4.5
1213          E045173    3000  8FBCU32     0.00   730.76     0.00          6.0
1214          E047758    1000  8FBCU32     0.00   501.97   173.01          6.0
1215          E048058    1000  8FBE20U     0.00   49

,Equipment,KOM Bin,Model,Total Damage,Total PM,Total Repair,Total Actual Hours
0,38446,4000,7FBCU15,731.60,0.00,0.00,3.3
1,38446,5000,7FBCU15,972.24,1538.94,1496.18,29.9
2,38446,6000,7FBCU15,1106.35,2164.87,3624.66,49.6
3,38446,7000,7FBCU15,0.00,1969.76,3594.30,52.6
4,38446,8000,7FBCU15,0.00,140.58,2137.28,15.3


In [15]:
# Linear regression model on dataset using scikit package
model, pred_df, lr_score = linreg(toyo_bin_df)

 
R2 of Linear Regression Model:  0.8726
 
Intercept:  -248.3695756884922
 
Coefficients:  [ 1.19723314e-02 -7.14343239e-01 -1.11021989e+00  1.41161830e+02]
 
Linear Regression Equation:
  Repair = -248.3696 + (0.0120 * KOM Bin) + (-0.7143 * Damage) + (-1.1102 * PM) + (141.1618 * Actual Hours)


In [16]:
# Linear regression model on dataset using statsmodel package
olsreg(toyo_bin_df)

                            OLS Regression Results                            
Dep. Variable:           Total Repair   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     946.5
Date:                Fri, 26 Jul 2024   Prob (F-statistic):          1.58e-310
Time:                        19:18:57   Log-Likelihood:                -6661.8
No. Observations:                 851   AIC:                         1.333e+04
Df Residuals:                     846   BIC:                         1.336e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -263.0466     45

# <span style='color:#f1552e'>  Interactive Plot & Predictions

## Economic Life Prediction Plot

In [17]:
interact(plot, Equipment=pred_df["Equipment"].unique(),  Acquistion = (30000, 42000, 1000));

interactive(children=(Dropdown(description='Equipment', options=('38446', '40306', '40307', '40309', '40310', …

## Predictions

In [18]:
display(HTML("<h1> <span style='color:#f1552e'> <center> Repair Cost Prediction"), Box, output)

Output()

In [ ]:
pip install explainerdashboard

## Explainer Dashboard

In [21]:
import explainerdashboard
from explainerdashboard import RegressionExplainer, ExplainerDashboard

X = pred_df[["KOM Bin", "Total Damage", "Total PM", "Total Actual Hours"]]
y = pred_df["Actual Repair"]

ls = LinearRegression().fit(X, y)

explainer = RegressionExplainer(ls, X, y)
ExplainerDashboard(explainer).run()

Generating self.shap_explainer = shap.LinearExplainer(modelX)...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating dependencies...
Calculating importances...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.1.23:8050


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_4b62e2d8-311b-4517-bf82-d066213cac3a (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002B056BAE4C0>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))